# Test ARMA-GARCH model

In [1]:
import fynance as fy
from fynance.tools import metrics
import numpy as np

In [2]:
from fynance.models import econometric_models as em

In [ ]:
em.estimation

In [3]:
X = (2 * np.random.rand(100000) - 1) * 0.01 + 0.001
X = np.cumprod(X + 1)

In [4]:
#print(metrics.sharpe_py(X, period=252))
print(metrics.sharpe_cy(X, period=252))

3.0764420819189944


In [5]:
metrics.roll_sharpe_cy(X)

array([0.        , 0.        , 6.79157014, ..., 3.07614194, 3.07645279,
       3.07644208])

In [7]:
metrics.roll_sharpe_cy2(X)

array([0.        , 0.        , 4.2380812 , ..., 3.08043716, 3.08034817,
       3.08053149])

In [5]:
metrics.roll_sharpe_cy3(X)

array([ 0.        , 20.60535755,  5.07775599, ...,  3.08045209,
        3.08036311,  3.08054642])

In [45]:
%timeit metrics.roll_sharpe_cy(X)

12.2 s ± 572 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
%timeit metrics.roll_sharpe_cy2(X)

30.7 s ± 158 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit metrics.roll_sharpe_cy3(X)

11.5 s ± 68.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
np.std(X)

0.27679332278812685

In [7]:
def sharpe_v2(series, period=252):
    """ 
    Function to compute the total return over the volatility, known as the 
    Sharpe ratio.
    
    Parameters
    ----------
    series: numpy.ndarray(dim=1, dtype=float)
        Prices of the index.
    period: int (default: 252)
        Number of period per year.

    Returns
    -------
    Float, it's the Sharpe ratio.
    """

    T = series.size 
    ret_vect = series[1:] / series[:-1] - 1
    annual_return = np.float_power(series[-1] / series[0], period / T) - 1
    vol = sqrt(period) * np.std(ret_vect)

    if vol == 0.:
        return 0.
    return annual_return / vol

In [53]:
%timeit sharpe_v2(X)

34.8 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
from math import sqrt

In [11]:
sharpe_v2(X, period=252)

-0.20415323857920192

In [12]:
metrics.sharpe_py(X)

-0.20415323857920192

In [10]:
np.arange(0, 5, dtype=iter)

TypeError: data type not understood

In [45]:
def roll_sharpe(series, period=252, win=0):
    """
    roll_sharpe(series, period=252, win=0)
    
    Vectorized rolling sharpe 
    Parameters
    ----------
    series: np.ndarray[dtype=np.float64, ndim=1]
        Financial series of prices or indexed values.
    period: int (default 252)
        Number of period in a year.
    win: int (default 0)
        Size of the rolling window. If less of two, 
    rolling sharpe is compute on all the past.
    
    Returns
    -------
    rolling_sharpe: np.ndarray[np.float64, ndim=1]
    """
    T = len(series)
    t = np.arange(1, T + 1)
    ret_vect = np.zeros([T])
    ret_vect[1: ] = series[1: ] / series[: -1] - 1
    # Compute rolling cumulative returns
    ret_cum = series / series[0] 
    if win > 1:
        ret_cum[win: ] = series[win: ] / series[: -win]
    # Compute rolling mean
    ret_mean = np.cumsum(ret_vect) / t
    if win > 1:
        ret_mean[win: ] = (ret_mean[win: ] * t[win: ] - ret_mean[: -win] * t[: -win]) / win
    # Compute rolling volatility
    ret_vol = np.cumsum(np.square(ret_vect - ret_mean)) / t
    if win > 1:
        ret_vol[win: ] = (ret_vol[win: ] * t[win: ] - ret_vol[: -win] * t[: -win]) / win
    ret_vol[ret_vol == 0] = 1e10
    # Compute rolling sharpe
    if win > 1:
        t[win: ] = win
    return (np.float_power(ret_cum, period / t) - 1.) / (np.sqrt(period * ret_vol))

In [10]:
%timeit metrics.roll_sharpe(X)

9.72 ms ± 196 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
metrics.roll_sharpe(X, win=1)

array([  0.        , -15.29227881, -18.462136  , ...,   3.09730861,
         3.09741683,   3.09748274])

In [29]:
ret_vect = np.zeros([X.size])
ret_vect[1: ] = X[1: ] / X[: -1] - 1
np.var(ret_vect)

3.3375942887959074e-05

In [34]:
m = np.cumsum(ret_vect) / np.arange(1, X.size + 1)
s = np.cumsum(np.square(ret_vect - m)) / np.arange(1, X.size + 1)

In [35]:
s

array([0.00000000e+00, 2.01211116e-06, 3.55256670e-06, ...,
       3.33725708e-05, 3.33723195e-05, 3.33722466e-05])

In [36]:
m

array([0.        , 0.00200605, 0.00071827, ..., 0.00100454, 0.00100451,
       0.00100456])